In [1]:
# import os
# import sys
# import time as t
# import datetime as dt

# import numpy as np
# import pandas as pd

# import matplotlib.pyplot as plt
# from tabulate import tabulate

# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_error, mean_squared_error

# import tensorflow as tf
# from tensorflow import keras

# # 项目内自定义模块
# base_dir = r"D:\Downloads\lstm-load-forecasting-master\lstm-load-forecasting-master"
# sys.path.append(os.path.join(base_dir))                # 确保能 import lstm_load_forecasting
# from lstm_load_forecasting import lstm                 # 你刚改好的 lstm.py

# print("TensorFlow:", tf.__version__)

# # ========= 模型类别 & 特征 =========
# model_cat_id = "NF01"

# # 这里用到的数据列：target = actual，其它为输入特征
# feature_cols = [
#     'hour_of_day', 'day_index',
#     'n_packets', 'n_bytes',
#     'n_dest_asn', 'n_dest_ports', 'n_dest_ip',
#     'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes',
#     'dir_ratio_packets', 'dir_ratio_bytes',
#     'avg_duration', 'avg_ttl'
# ]
# target_col = 'actual'

# # ========= 时间窗口 & 超参数搜索空间 =========
# TIMESTEPS  = 60            # 用过去 60 分钟的序列预测下一分钟
# timesteps  = [TIMESTEPS]

# layer_conf = [True, True]  # 两层 LSTM 都启用
# cells      = [[32, 64]]    # 第一层 32 单元，第二层 64 单元
# dropout    = [0.0, 0.1]    # 两种 dropout 配置
# batch_size = [64]          # 批大小（影响训练/评估）
# early_stopping = True
# validation_split = 0.2

# EPOCHS     = 20
# MIN_DELTA  = 0.002
# PATIENCE   = 3

# # 结果汇总 DataFrame
# results = pd.DataFrame(columns=[
#     'model_name', 'config', 'dropout',
#     'train_loss', 'train_rmse', 'train_mae', 'train_mape',
#     'valid_loss', 'valid_rmse', 'valid_mae', 'valid_mape',
#     'test_rmse',  'test_mae',  'test_mape',
#     'epochs', 'batch_train', 'input_shape',
#     'total_time', 'time_step', 'splits'
# ])


In [2]:
import os
import sys
import time as t
import datetime as dt

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tabulate import tabulate

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow import keras

# 项目内自定义模块
base_dir = r"D:\Downloads\lstm-load-forecasting-master\lstm-load-forecasting-master"
sys.path.append(os.path.join(base_dir))                
from lstm_load_forecasting import lstm

print("TensorFlow:", tf.__version__)

# ========= 模型类别 & 特征 =========
model_cat_id = "NF01"

feature_cols = [
    'hour_of_day', 'day_index',
    'n_packets', 'n_bytes',
    'n_dest_asn', 'n_dest_ports', 'n_dest_ip',
    'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes',
    'dir_ratio_packets', 'dir_ratio_bytes',
    'avg_duration', 'avg_ttl'
]
target_col = 'actual'

# ========= ⚠️ 强制修正配置 (目标：40个) ⚠️ =========
TIMESTEPS  = 60            
# 1. 时间步长 (4种)
timesteps_list  = [30, 60, 90, 120] 

# 2. LSTM 层结构 (关键修改：用双层列表包裹，确保只算作1种结构)
# 之前的 [True, True] 可能被误读为 "Option1: True, Option2: True" 导致翻倍
layer_conf_list = [[True, True]] 

# 3. LSTM 单元配置 (5种)
cells_list      = [
    [32, 32],      
    [64, 32],      
    [64, 64],      
    [128, 64],     
    [128, 128]     
]

# 4. Dropout 配置 (2种)
dropout_list    = [0.0, 0.2]

# 5. 批大小 (1种)
batch_size_list = [64]

# ========= 训练控制参数 =========
early_stopping   = True
validation_split = 0.2   

EPOCHS     = 20
MIN_DELTA  = 0.002
PATIENCE   = 3

# 结果汇总 DataFrame
results = pd.DataFrame(columns=[
    'model_name', 'config', 'dropout',
    'train_loss', 'train_rmse', 'train_mae', 'train_mape',
    'valid_loss', 'valid_rmse', 'valid_mae', 'valid_mape',
    'test_rmse',  'test_mae',  'test_mape',
    'epochs', 'batch_train', 'input_shape',
    'total_time', 'time_step', 'splits'
])

TensorFlow: 2.15.0


In [3]:
# ===== 读取原始数据 & 构造特征 =====
data_path = os.path.join(base_dir, "data", "11minutes.csv")

df_raw = pd.read_csv(data_path)

print("Raw shape:", df_raw.shape)
print("Columns:", df_raw.columns)

# 目标列（你的是 n_flows）
target_col = 'n_flows'
df_raw['actual'] = df_raw[target_col].astype(float)

# 特征列（你的真实特征）
feature_cols = [
    'n_packets', 'n_bytes',
    'n_dest_asn', 'n_dest_ports', 'n_dest_ip',
    'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes',
    'dir_ratio_packets', 'dir_ratio_bytes',
    'avg_duration', 'avg_ttl'
]

X_full = df_raw[feature_cols].copy()
y_full = df_raw['actual'].copy()

# ===== 按时间顺序划分 train/test（比如前 80% 做训练）=====
n_total = len(df_raw)
split_idx = int(n_total * 0.8)
X_train_raw, X_test_raw = X_full.iloc[:split_idx], X_full.iloc[split_idx:]
y_train_raw, y_test_raw = y_full.iloc[:split_idx], y_full.iloc[split_idx:]

print("Train size:", len(X_train_raw), "Test size:", len(X_test_raw))

# ===== 分别对 X 和 y 做标准化（方便以后单独反归一化）=====
scaler_X = StandardScaler()
X_train = pd.DataFrame(
    scaler_X.fit_transform(X_train_raw),
    index=X_train_raw.index,
    columns=X_train_raw.columns
)
X_test = pd.DataFrame(
    scaler_X.transform(X_test_raw),
    index=X_test_raw.index,
    columns=X_test_raw.columns
)

scaler_y = StandardScaler()
y_train = pd.Series(
    scaler_y.fit_transform(y_train_raw.values.reshape(-1,1)).reshape(-1),
    index=y_train_raw.index,
    name='actual'
)
y_test = pd.Series(
    scaler_y.transform(y_test_raw.values.reshape(-1,1)).reshape(-1),
    index=y_test_raw.index,
    name='actual'
)

print("After scaling: X_train", X_train.shape, "y_train", y_train.shape)


Raw shape: (40298, 13)
Columns: Index(['id_time', 'n_flows', 'n_packets', 'n_bytes', 'n_dest_asn',
       'n_dest_ports', 'n_dest_ip', 'tcp_udp_ratio_packets',
       'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes',
       'avg_duration', 'avg_ttl'],
      dtype='object')
Train size: 32238 Test size: 8060
After scaling: X_train (32238, 11) y_train (32238,)


In [4]:
# ===== 输出目录 & 模型组合 =====
res_dir   = os.path.join(base_dir, f"{model_cat_id}_results/")
plot_dir  = os.path.join(base_dir, f"{model_cat_id}_plots/")
model_dir = os.path.join(base_dir, f"{model_cat_id}_models/")

os.makedirs(res_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

output_table = os.path.join(
    res_dir, model_cat_id + "_results_" + t.strftime("%Y%m%d") + ".csv"
)
test_output_table = os.path.join(
    res_dir, model_cat_id + "_test_results_" + t.strftime("%Y%m%d") + ".csv"
)

# # 生成模型组合
# models = lstm.generate_combinations(
#     model_name=model_cat_id + "_",
#     layer_conf=layer_conf,
#     cells=cells,
#     dropout=dropout,
#     batch_size=batch_size,
#     timesteps=timesteps
# )

# print("Number of model configs generated:", len(models))

# 生成模型组合
models = lstm.generate_combinations(
    model_name=model_cat_id + "_",
    layer_conf=layer_conf_list,  # 使用新变量名
    cells=cells_list,            # 使用新变量名
    dropout=dropout_list,        # 使用新变量名
    batch_size=batch_size_list,  # 使用新变量名
    timesteps=timesteps_list     # 使用新变量名
)

# ⚠️ 强制截断：无论生成多少，只取前 40 个
# 这能防止 256 个模型把电脑跑死
if len(models) > 40:
    print(f"检测到 {len(models)} 个组合，正在强制截断为 40 个...")
    models = models[:40]

print("Number of model configs generated:", len(models))


| Number of model configs generated | 256 |
检测到 256 个组合，正在强制截断为 40 个...
Number of model configs generated: 40


In [ ]:
# ===== 训练所有模型（多配置搜索） =====

MAX_MODELS   = len(models)      # 想先试少一点可以改小
EPOCHS       = EPOCHS
PATIENCE     = PATIENCE
MIN_DELTA    = MIN_DELTA
SAVE_MODELS  = True
FLUSH_EVERY  = 2

epochs   = EPOCHS
patience = PATIENCE
min_delta = MIN_DELTA

models = models[:MAX_MODELS]

def pick(h, *keys):
    for k in keys:
        if k in h:
            return h[k]
    raise KeyError(f"history 中没有 {keys}，可用键：{list(h.keys())}")

start_time   = t.time()
pending_rows = []

for idx, m in enumerate(models, 1):
    stopper = t.time()
    print('========================= Model {}/{} ========================='.format(idx, len(models)))
    print(tabulate(
        [['Starting with model', m['name']], ['Starting time', dt.datetime.fromtimestamp(stopper)]],
        tablefmt="jira", numalign="right", floatfmt=".3f"
    ))
    try:
        # 1) 构建模型
        model = lstm.create_model(
            layers=m['layers'],
            timesteps=m['timesteps'],
            features=X_train.shape[1]
        )

        # 2) 训练：用序列窗口 rearrange=True
        history = lstm.train_model(
            model=model,
            mode='fit',
            y=y_train.values,
            X=X_train.values,
            batch_size=m['batch_size'],
            timesteps=m['timesteps'],
            epochs=epochs,
            rearrange=True,
            validation_split=validation_split,
            verbose=1,
            early_stopping=early_stopping,
            min_delta=min_delta,
            patience=patience
        )

        h = history.history
        val_loss_hist   = pick(h, 'val_loss')
        train_loss_hist = pick(h, 'loss')
        train_mae_hist  = pick(h, 'mae', 'mean_absolute_error')
        val_mae_hist    = pick(h, 'val_mae', 'val_mean_absolute_error')

        min_idx   = int(np.argmin(val_loss_hist))
        min_epoch = min_idx + 1

        row = {
            'model_name': m['name'],
            'config': m,
            'dropout': m['layers'][0].get('dropout', 0),
            'train_loss': float(train_loss_hist[min_idx]),
            'train_rmse': 0,
            'train_mae' : float(train_mae_hist[min_idx]),
            'train_mape': 0,
            'valid_loss': float(val_loss_hist[min_idx]),
            'valid_rmse': 0,
            'valid_mae' : float(val_mae_hist[min_idx]),
            'valid_mape': 0,
            'test_rmse': 0, 'test_mae': 0, 'test_mape': 0,
            'epochs': f'{min_epoch}/{epochs}',
            'batch_train': m['batch_size'],
            'input_shape': (m['timesteps'], X_train.shape[1]),
            'total_time': t.time() - stopper,
            'time_step': m['timesteps'],
            'splits': f"0-{len(X_train)}"
        }
        pending_rows.append(row)

        if SAVE_MODELS:
            model_path = os.path.join(model_dir, f"{m['name']}.h5")
            model.save(model_path)

        if (idx % FLUSH_EVERY == 0) or (idx == len(models)):
            if pending_rows:
                results = pd.concat([results, pd.DataFrame(pending_rows)], ignore_index=True)
                results.to_csv(output_table, sep=';', index=False)
                pending_rows = []

    except BaseException as e:
        print('=============== ERROR {}/{} ============='.format(idx, len(models)))
        print(tabulate(
            [['Model:', m['name']], ['Config:', m]],
            tablefmt="jira", numalign="right", floatfmt=".3f"
        ))
        print('Error:', e)
        pending_rows.append({'model_name': m['name'], 'config': m, 'train_loss': str(e)})
    finally:
        try:
            keras.backend.clear_session()
        except:
            pass
        try:
            del model
        except:
            pass

if pending_rows:
    results = pd.concat([results, pd.DataFrame(pending_rows)], ignore_index=True)
    results.to_csv(output_table, sep=';', index=False)

print("Done. Results saved to:", output_table)


========================= Model 1/40 =========================
| Starting with model | NF01_1_l-32_l-64_l-64_l-128_l-128 |
| Starting time       | 2025-12-03 18:15:05.876807        |


Epoch 1/20


403/403 [==============================] - 47s 96ms/step - loss: 0.2632 - mae: 0.3836 - val_loss: 0.7873 - val_mae: 0.7366
Epoch 2/20
403/403 [==============================] - 41s 101ms/step - loss: 0.1508 - mae: 0.2791 - val_loss: 0.4559 - val_mae: 0.5157
Epoch 3/20
403/403 [==============================] - 41s 101ms/step - loss: 0.1170 - mae: 0.2433 - val_loss: 0.3428 - val_mae: 0.4384
Epoch 4/20
403/403 [==============================] - 41s 103ms/step - loss: 0.0968 - mae: 0.2220 - val_loss: 0.3066 - val_mae: 0.4118
Epoch 5/20
403/403 [==============================] - 43s 106ms/step - loss: 0.0891 - mae: 0.2133 - val_loss: 0.3140 - val_mae: 0.4088
Epoch 6/20
403/403 [==============================] - 41s 101ms/step - loss: 0.0887 - mae: 0.2136 - val_loss: 0.2832 - val_mae: 0.3832
Epoc

D:\Anaconda\envs\network_proj_new\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/20
403/403 [==============================] - 76s 170ms/step - loss: 0.2542 - mae: 0.3758 - val_loss: 0.4383 - val_mae: 0.5089
Epoch 2/20
403/403 [==============================] - 68s 168ms/step - loss: 0.1506 - mae: 0.2797 - val_loss: 0.3007 - val_mae: 0.4119
Epoch 3/20
403/403 [==============================] - 65s 162ms/step - loss: 0.1177 - mae: 0.2453 - val_loss: 0.2584 - val_mae: 0.3819
Epoch 4/20
403/403 [==============================] - 65s 162ms/step - loss: 0.0971 - mae: 0.2223 - val_loss: 0.2573 - val_mae: 0.3850
Epoch 5/20
403/403 [==============================] - 67s 166ms/step - loss: 0.0894 - mae: 0.2139 - val_loss: 0.2501 - val_mae: 0.3735
Epoch 6/20
403/403 [==============================] - 72s 179ms/step - loss: 0.0881 - mae: 0.2126 - val_loss: 0.2474 - val_mae: 0.3696
Epoch 7/20
403/403 [==============================] - 72s 178ms/step - loss: 0.0874 - mae: 0.2117 - val_loss: 0.2548 - val_mae: 0.3754
Epoch 8/20
403/403 [==============================] - 6

C:\UserData\AppData\Local\Temp\ipykernel_21460\1135675434.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame(pending_rows)], ignore_index=True)


Epoch 1/20
402/402 [==============================] - 119s 278ms/step - loss: 0.2566 - mae: 0.3776 - val_loss: 0.6377 - val_mae: 0.6629
Epoch 2/20
402/402 [==============================] - 112s 279ms/step - loss: 0.1531 - mae: 0.2824 - val_loss: 0.4716 - val_mae: 0.5356
Epoch 3/20
402/402 [==============================] - 111s 275ms/step - loss: 0.1200 - mae: 0.2460 - val_loss: 0.3942 - val_mae: 0.4891
Epoch 4/20
402/402 [==============================] - 100s 248ms/step - loss: 0.1004 - mae: 0.2262 - val_loss: 0.3602 - val_mae: 0.4686
Epoch 5/20
402/402 [==============================] - 110s 273ms/step - loss: 0.0908 - mae: 0.2155 - val_loss: 0.3345 - val_mae: 0.4424
Epoch 6/20
402/402 [==============================] - 110s 275ms/step - loss: 0.0887 - mae: 0.2135 - val_loss: 0.3137 - val_mae: 0.4233
Epoch 7/20
402/402 [==============================] - 109s 270ms/step - loss: 0.0884 - mae: 0.2132 - val_loss: 0.2997 - val_mae: 0.4133
Epoch 8/20
402/402 [============================

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



403/403 [==============================] - 82s 203ms/step - loss: 0.1669 - mae: 0.2985 - val_loss: 0.3967 - val_mae: 0.4823
Epoch 3/20
403/403 [==============================] - 79s 196ms/step - loss: 0.1363 - mae: 0.2656 - val_loss: 0.2916 - val_mae: 0.3987
Epoch 4/20
127/403 [========>.....................] - ETA: 46s - loss: 0.1464 - mae: 0.2665

In [ ]:
# ===== 模型选择：按验证集 valid_mae 选 Top-5 =====
import glob

pattern    = os.path.join(res_dir, f"{model_cat_id}_results_*.csv")
candidates = sorted(glob.glob(pattern))

if len(candidates) == 0:
    results_csv = results.copy()
    print("⚠ 没找到结果文件，使用当前内存里的 results")
else:
    results_fn  = candidates[-1]
    print("使用结果文件:", results_fn)
    results_csv = pd.read_csv(results_fn, delimiter=';')

print("列名:", list(results_csv.columns))

if 'valid_mae' not in results_csv.columns:
    raise ValueError(f"'valid_mae' 列不存在，可用列: {list(results_csv.columns)}")

top_models = results_csv.dropna(subset=['valid_mae']).nsmallest(5, 'valid_mae')
top_models = top_models.reset_index(drop=True)
print("Top-5 models:")
display(top_models[['model_name', 'valid_mae', 'train_mae', 'epochs']])


In [ ]:
from tensorflow.keras.models import load_model

test_rows   = []
predictions = {}

for i, row in top_models.iterrows():
    model_name = row['model_name']
    print(f">>> [{i+1}/{len(top_models)}] Evaluating {model_name}")

    base = os.path.join(model_dir, model_name)
    filename = base + ".keras" if os.path.exists(base + ".keras") else base + ".h5"
    if not os.path.exists(filename):
        print("  [WARN] 模型文件不存在:", filename)
        continue

    model = load_model(filename)

    batch_size_eval = int(row.get('batch_train', 64))
    t_steps = int(row.get('time_step', TIMESTEPS))

    # 用同一套序列化逻辑，确保长度一致
    X_seq, y_seq = lstm.make_sequences(X_test.values, y_test.values, t_steps)

    max_batch = len(X_seq) // batch_size_eval
    X_eval = X_seq[:max_batch * batch_size_eval]
    y_eval = y_seq[:max_batch * batch_size_eval]

    print("  X_eval:", X_eval.shape, "y_eval:", y_eval.shape)

    pred = model.predict(X_eval, batch_size=batch_size_eval, verbose=0)
    pred = pred.reshape(-1)
    y_eval = y_eval.reshape(-1)

    mae = mean_absolute_error(y_eval, pred)
    mse = mean_squared_error(y_eval, pred)

    test_rows.append({
        "Model name": model_name,
        "Mean squared error": mse,
        "Mean absolute error": mae
    })

    predictions[model_name] = {
        "pred_scaled": pred.copy(),
        "y_scaled": y_eval.copy()
    }

    keras.backend.clear_session()
    del model

test_results = pd.DataFrame(test_rows).sort_values("Mean absolute error")
test_results = test_results.set_index("Model name")
print("\nTest performance:")
print(tabulate(test_results, headers="keys", tablefmt="grid", numalign="right", floatfmt=".4f"))

test_results.to_csv(test_output_table, sep=';', index=True)
print("Saved test results to:", test_output_table)

best_model_name = test_results.index[0]
print("\nBest model on test set:", best_model_name)


In [ ]:
# ===== 最佳模型预测 & 异常检测 =====
best_model_file = os.path.join(model_dir, best_model_name + ".h5")
best_model = load_model(best_model_file)

batch_size_best = int(top_models.loc[top_models['model_name'] == best_model_name, 'batch_train'].iloc[0])
t_steps_best    = int(top_models.loc[top_models['model_name'] == best_model_name, 'time_step'].iloc[0])

# 序列化测试集
X_seq, y_seq = lstm.make_sequences(X_test.values, y_test.values, t_steps_best)
max_batch = len(X_seq) // batch_size_best
X_eval = X_seq[:max_batch * batch_size_best]
y_eval = y_seq[:max_batch * batch_size_best]

pred_scaled = best_model.predict(X_eval, batch_size=batch_size_best, verbose=0).reshape(-1)
y_scaled    = y_eval.reshape(-1)

# 反归一化回原始单位
y_true = scaler_y.inverse_transform(y_scaled.reshape(-1,1)).reshape(-1)
y_pred = scaler_y.inverse_transform(pred_scaled.reshape(-1,1)).reshape(-1)

# 计算残差
residuals = np.abs(y_true - y_pred)

print("原始尺度 MAE:", mean_absolute_error(y_true, y_pred))
print("原始尺度 MSE:", mean_squared_error(y_true, y_pred))

# ===== 简单异常检测：按残差做阈值 =====
# 用均值 + 3*std 作为阈值，或者 99 分位数
thr_mean_std = residuals.mean() + 3 * residuals.std()
thr_quantile = np.quantile(residuals, 0.99)

print("Threshold (mean+3std):", thr_mean_std)
print("Threshold (99% quantile):", thr_quantile)

# 选一个你喜欢的阈值（这里用 quantile）
threshold = thr_quantile

anomaly_flag = residuals > threshold

# 对齐到原始索引：序列化会从第 t_steps 开始
idx_eval = y_test.index[t_steps_best : t_steps_best + len(y_true)]

anomaly_df = pd.DataFrame({
    "minute_index": idx_eval,
    "y_true": y_true,
    "y_pred": y_pred,
    "residual": residuals,
    "is_anomaly": anomaly_flag.astype(int)
})
anomaly_df = anomaly_df.set_index("minute_index")

anomaly_path = os.path.join(
    res_dir, f"{best_model_name}_anomalies_{t.strftime('%Y%m%d')}.csv"
)
anomaly_df.to_csv(anomaly_path, sep=';')
print("Anomalies saved to:", anomaly_path)

# 简单画个图看看
plt.figure(figsize=(12,4))
plt.plot(idx_eval, y_true, label="actual")
plt.plot(idx_eval, y_pred, label="pred")
plt.scatter(idx_eval[anomaly_flag], y_true[anomaly_flag], marker='x', label="anomaly")
plt.legend()
plt.title("Best model prediction & anomalies")
plt.show()


In [ ]:
import time

print("开始回放测试集，模拟在线预警（按时间顺序）...\n")
print("格式：时间索引 actual  pred  residual   是否告警")

for i, idx in enumerate(idx_eval):
    real = y_true[i]
    pred = y_pred[i]
    res  = residuals[i]
    is_anom = res > threshold  # 用你前面算的阈值（比如 99% 分位数）

    flag = "  <-- ⚠ ALERT 异常流量" if is_anom else ""
    print(f"{idx:6d}  {real:8.0f}  {pred:8.0f}  {res:8.0f}{flag}")

    # 为了演示效果慢一点，可以调节速度
    time.sleep(0.05)   # 50ms 一条，课堂上看着比较“流动”
